In [1]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import wandb
import numpy as np
import os
import pandas as pd
import zipfile
import random
from tqdm import tqdm
from google.colab import files
from IPython.display import display
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define the model architecture
class Encoder(nn.Module):
    def __init__(self, input_size,embedding_size, hidden_size,batch_size,encoder_num_layers, cell_type, bidirectional, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.batch_size=batch_size
        self.embedding_size=embedding_size
        self.encoder_num_layers=encoder_num_layers
        self.cell_type = cell_type
        self.bidirectional=bidirectional



        if cell_type == "RNN":
            self.rnn = nn.RNN(self.embedding_size, self.hidden_size,self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.encoder_num_layers, bidirectional=bidirectional, dropout=dropout)




    def forward(self, input_seq, hidden):
        embedded = self.dropout((self.embedding(input_seq.long())).view(-1,self.batch_size, self.embedding_size))
        outputs, hidden = self.rnn(embedded,hidden)


        if self.bidirectional:
            if self.cell_type == "LSTM":
                # Dividing the hidden state into parts for each direction
                hidden_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)
                cell_state = hidden[0].view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

                # Combining the hidden and cell states by taking their average
                hidden = (torch.add(hidden_state[0], hidden_state[1]) / 2, torch.add(cell_state[0], cell_state[1]) / 2)
            else:
                # Dividing the hidden state into parts for each direction
                hidden = hidden.view(2, self.encoder_num_layers, self.batch_size, self.hidden_size)

                # Combining the hidden states by taking their average
                hidden = torch.add(hidden[0], hidden[1]) / 2

            # Splitting the output tensor into parts for each direction
            split_tensor = torch.split(outputs, self.hidden_size, dim=-1)

            # Combining the outputs by taking their average
            output = torch.add(split_tensor[0], split_tensor[1]) / 2


        return outputs, hidden



    def initHidden(self):
            num_directions = 2 if self.bidirectional else 1  # For bidirectional, set to 2, otherwise 1
            if self.cell_type == "LSTM":
                return (torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device),
                        torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device))
            else:
                return torch.zeros(self.encoder_num_layers * num_directions, self.batch_size, self.hidden_size, device=device)

class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, batch_size, decoder_num_layers, cell_type, dropout, MAX_LENGTH):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.decoder_num_layers = decoder_num_layers
        self.MAX_LENGTH = MAX_LENGTH
        self.cell_type = cell_type
        self.dropout=dropout

        self.embedding = nn.Embedding(output_size, embedding_size)
        self.dropout = nn.Dropout(dropout)
        if cell_type == "RNN":
            self.rnn = nn.RNN(embedding_size, hidden_size, self.decoder_num_layers,dropout=dropout)
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(embedding_size, hidden_size, self.decoder_num_layers, dropout=dropout)


        self.out = nn.Linear(hidden_size, output_size)

        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output = self.embedding(input.long()).view(-1,self.batch_size, self.embedding_size)

        output = F.relu(output)

        output, hidden = self.rnn(output, hidden)

        output = self.softmax(self.out(output))
        return output, hidden




In [4]:
def Datasetretrival(pathofzip, folder_name):
    # Define the path to your zip file
    zip_file_path = pathofzip

    # Extract the contents of the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('extracted_data')

    # Define the path to the extracted data directory
    extracted_data_dir = 'extracted_data/aksharantar_sampled'
    contents = os.listdir(extracted_data_dir)

    # Initialize empty lists for train, test, and validation datasets
    train_datasets = []
    test_datasets = []
    val_datasets = []

 # Load train, test, and validation CSV files from the specified folder
    folder_path = os.path.join(extracted_data_dir, folder_name)

    # List all files in the folder
    folder_files = os.listdir(folder_path)

    # Filter files with the specified folder name as prefix
    foldername_prefix = folder_name + "_"
    folder_files_with_prefix = [file for file in folder_files if file.startswith(foldername_prefix)]

    for file in folder_files_with_prefix:
        file_path = os.path.join(folder_path, file)
        if 'train' in file:
            train_datasets.append(pd.read_csv(file_path,header=None))
        elif 'test' in file:
            test_datasets.append(pd.read_csv(file_path,header=None))
        elif 'val' in file:
            val_datasets.append(pd.read_csv(file_path,header=None))

    # Concatenate the loaded dataframes to create single train, test, and validation datasets
    train_dataset = pd.concat(train_datasets, ignore_index=True)
    test_dataset = pd.concat(test_datasets, ignore_index=True)
    val_dataset = pd.concat(val_datasets, ignore_index=True)


    return train_dataset, val_dataset, test_dataset

In [5]:
def vecorizeddata(data_pairs,index2char, char2index):

    # Tokens
    # 0 -> SOS
    # 1 -> EOS
    # 3 -> Pad

    maxlength_input = 0
    maxlength_output = 0


    # Adding in the main index2char and char2index dictionary
    for word_pair in data_pairs:
        maxlength_input = max(maxlength_input, len(word_pair[0]))

        for char in word_pair[0]:
            if char not in  char2index:
                char2index[char] = len(char2index)
                index2char[len(index2char)] = char



        maxlength_output = max(maxlength_output, len(word_pair[1]))

        for char in word_pair[1]:
            if char not in  char2index:
                char2index[char] = len(char2index)
                index2char[len(index2char)] = char


    MAX_LENGTH = max(maxlength_input, maxlength_output) + 2

    max_of_all = max(maxlength_input, maxlength_output)

    SOS_token = 0
    EOS_token = 1
    PAD_token = 2

    vec_pair_list = []
    for word_pair in data_pairs:


        vec_1 = []
        for char in word_pair[0]:
            vec_1.append(char2index[char])

        wordvec_1 = vec_1
        wordvec_1.append(EOS_token)

        for i in range(MAX_LENGTH - len(word_pair[0])):
            wordvec_1.append(PAD_token)
        wordvec_1 = torch.LongTensor(wordvec_1)
        eng_vec = wordvec_1


        vec_2 = []
        for char in word_pair[1]:
            vec_2.append(char2index[char])

        wordvec_2 = vec_2
        wordvec_2.append(EOS_token)

        for i in range(MAX_LENGTH - len(word_pair[1])):
            wordvec_2.append(PAD_token)
        wordvec_2 = torch.LongTensor(wordvec_2)
        guj_vec = wordvec_2


        vec_pair = (eng_vec, guj_vec)
        vec_pair_list.append(vec_pair)

    return vec_pair_list,char2index, index2char ,MAX_LENGTH


In [7]:
def train(encoder, decoder,train_loader, encoder_optimizer,decoder_optimizer,encoder_num_layers,decoder_num_layers,cell_type,criterion,char2index,index2char,MAX_LENGTH,teacher_forcing_ratio,attention,beam_width, device):
    total_loss = 0
    correct=0
    total=0
    encoder.to(device)
    decoder.to(device)
    encoder.train()
    decoder.train()
    target_length=0


    for data in tqdm(train_loader):

        input_tensor, target_tensor = data
        input_tensor = input_tensor.to(device)
        target_tensor=target_tensor.to(device)
        batch_size = input_tensor.shape[0]
        input_tensor=input_tensor.T
        target_tensor=target_tensor.T

        encoder_hidden = encoder.initHidden()

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()


        input_length = len(input_tensor)
        target_length = len(target_tensor)


        encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)


        decoder_input = target_tensor[0]
        # Handle different numbers of layers in the encoder and decoder
        if encoder_num_layers != decoder_num_layers:
            if encoder_num_layers < decoder_num_layers:
                remaining_layers = decoder_num_layers - encoder_num_layers
                # Copy all encoder hidden layers and then repeat the top layer
                if cell_type == "LSTM":
                    top_layer_hidden = (encoder_hidden[0][-1].unsqueeze(0), encoder_hidden[1][-1].unsqueeze(0))
                    extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                    decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=0), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=0))
                else:
                    top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_size)
                    extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                    decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=0)

            else:
                # Slice the hidden states of the encoder to match the decoder layers
                if cell_type == "LSTM":
                    decoder_hidden = (encoder_hidden[0][-decoder_num_layers:], encoder_hidden[1][-decoder_num_layers:])
                else :
                    decoder_hidden = encoder_hidden[-decoder_num_layers:]
        else:
            decoder_hidden = encoder_hidden


        loss = 0

        pred=torch.zeros(len(target_tensor), batch_size).to(device)
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi=torch.squeeze(topi)
                pred[di]=topi

                decoder_input = target_tensor[di]


        else:

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi = torch.squeeze(topi)
                pred[di]=topi
                decoder_input = topi

        pred = pred.T
        act = target_tensor.T

        for i in range(len(pred)):
            f=0
            for j in range(len(pred[i])):
                if(pred[i][j]!=act[i][j]):
                    f=1
                    break
            if(f==0):
                correct += 1

        total_loss += loss
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()



    accuracy = correct / (len(train_loader) * batch_size )
    final_loss = total_loss /  (len(train_loader) * batch_size )
    return   final_loss , accuracy



In [8]:

# Define evaluation function
def evaluate(encoder, decoder, val_loader,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,attention,beam_width, device):
    EOS_token=1
    SOS_token=0
    correct=0
    total_loss=0
    target_length=0
    pred=[]
    predictions = []
    Input = []
    Target = []

    with torch.no_grad():

        encoder.eval()
        decoder.eval()
        for data in tqdm(val_loader):

            input_tensor, target_tensor = data
            batch_size = input_tensor.shape[0]
            input_tensor = input_tensor.to(device)
            target_tensor=target_tensor.to(device)




            input_tensor=input_tensor.T
            target_tensor=target_tensor.T

            encoder_hidden = encoder.initHidden()

            input_length = len(input_tensor)
            target_length = len(target_tensor)


            encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)


            decoder_input = target_tensor[0]
            # Handle different numbers of layers in the encoder and decoder
            if encoder_num_layers != decoder_num_layers:
                if encoder_num_layers < decoder_num_layers:
                    remaining_layers = decoder_num_layers - encoder_num_layers
                    # Copy all encoder hidden layers and then repeat the top layer
                    if cell_type == "LSTM":
                        top_layer_hidden = (encoder_hidden[0][-1].unsqueeze(0), encoder_hidden[1][-1].unsqueeze(0))
                        extra_hidden = (top_layer_hidden[0].repeat(remaining_layers, 1, 1), top_layer_hidden[1].repeat(remaining_layers, 1, 1))
                        decoder_hidden = (torch.cat((encoder_hidden[0], extra_hidden[0]), dim=0), torch.cat((encoder_hidden[1], extra_hidden[1]), dim=0))
                    else:
                        top_layer_hidden = encoder_hidden[-1].unsqueeze(0) #top_layer_hidden shape (1, batch_size, hidden_size)
                        extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
                        decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=0)

                else:
                    # Slice the hidden states of the encoder to match the decoder layers
                    if cell_type == "LSTM":
                        decoder_hidden = (encoder_hidden[0][-decoder_num_layers:], encoder_hidden[1][-decoder_num_layers:])
                    else :
                        decoder_hidden = encoder_hidden[-decoder_num_layers:]
            else:
                decoder_hidden = encoder_hidden


            loss = 0
            pred=torch.zeros(len(target_tensor), batch_size).to(device)


            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                decoder_output = torch.squeeze(decoder_output)
                loss += criterion(decoder_output, target_tensor[di].long())


                topv, topi = decoder_output.topk(1)
                topi = torch.squeeze(topi)
                pred[di]=topi
                decoder_input = topi

            pred = pred.T
            act = target_tensor.T
            act_eng = input_tensor.T
            for i in range(batch_size):
                pred_word=""
                input_word=""
                target_word = ""

                f=0
                for j in range(len(act[i])):

                    if(int(pred[i][j].item()) > 2):
                        pred_word += index2char[int(pred[i][j].item())]
                    if(int(act[i][j].item()) > 2):
                        target_word += index2char[int(act[i][j].item())]

                    if(pred[i][j]!=act[i][j]):
                        f=1

                if(f==0):
                    correct += 1

                for j in range(len(act_eng[i])):

                      if(int(act_eng[i][j].item()) > 2):
                          input_word += index2char[int(act_eng[i][j].item())]

                predictions.append(pred_word)
                Input.append(input_word)
                Target.append(target_word)


        total_loss += loss
        accuracy = correct / (len(val_loader) * batch_size )
        final_loss = total_loss / (len(val_loader) * batch_size )
    return  final_loss , accuracy , predictions ,Input , Target


In [9]:

# Define training function
def arguments(input_embedding_size,encoder_num_layers,decoder_num_layers,hidden_size,cell_type,bidirectional,batch_size,learning_rate,num_epochs,dropout,teacher_forcing_ratio,attention,beam_width,pathofzip,Folder_name):

    train_dataset ,val_dataset ,test_dataset= Datasetretrival(pathofzip,Folder_name)

    # Convert DataFrame to list of tuples
    data_pairs_train = [list(row) for row in train_dataset.values]
    data_pairs_val = [list(row) for row in val_dataset.values]
    data_pairs_test = [list(row) for row in test_dataset.values]



    index2char = {0:'<', 1: '>', 2 : '.'}
    char2index = {'<' : 0, '>' : 1, '.' : 2 }


    vec_pair_list_train,char2index, index2char,MAX_LENGTH_1=vecorizeddata(data_pairs_train,index2char, char2index)
    vec_pair_list_val,_,_,MAX_LENGTH_2=vecorizeddata(data_pairs_val,index2char, char2index)
    vec_pair_list_test,_,_,MAX_LENGTH_3=vecorizeddata(data_pairs_test,index2char, char2index)


    predictions = []
    Input = []
    Target = []


    MAX_LENGTH= 30
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader = DataLoader(vec_pair_list_train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(vec_pair_list_val, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(vec_pair_list_test, batch_size=batch_size, shuffle=False)

    # Define model
    encoder = Encoder(len(char2index),input_embedding_size, hidden_size,batch_size, encoder_num_layers, cell_type,bidirectional, dropout).to(device)
    decoder = Decoder(len(char2index),input_embedding_size, hidden_size,batch_size, decoder_num_layers, cell_type, dropout,MAX_LENGTH).to(device)
    #model = Seq2Seq(encoder, decoder, encoder_num_layers, decoder_num_layers,batch_size, hidden_size,bidirectional, cell_type,device).to(device)

    output_predictions=torch.tensor([]).to(device)
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)


    # Training loop
    for epoch in range(num_epochs):


        # Train the model
        train_loss, train_accuracy = train(encoder,decoder, train_loader, encoder_optimizer,decoder_optimizer,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,teacher_forcing_ratio,attention,beam_width, device)

        # Validation loop
        val_loss, val_accuracy,predictions ,Input , Target = evaluate(encoder,decoder, val_loader,encoder_num_layers,decoder_num_layers,cell_type, criterion,char2index,index2char,MAX_LENGTH,attention,beam_width, device)

        # Log metrics to Weights & Biases
        wandb.log({
            "Epoch": epoch + 1,
            "Train_Accuracy": train_accuracy,
            "Train_Loss": train_loss,
            "Val_Accuracy": val_accuracy,
            "Val_Loss": val_loss
        })

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs},\n Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f},\n Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}\n")

    print(f"\n Predictions Generated by current sweep and actual output:\n")
    dataframe = pd.DataFrame({"INPUT": Input, "PREDICTED": predictions,"ACTUAL":Target})
    dataframe.to_csv("predictions.csv", index=False)
    data = pd.read_csv("predictions.csv",header=None)
    #files.download("predictions.csv")
    display(data)



In [ ]:
#arguments(32,2,2,1024,"RNN",True,128,0.001,1,0.2,0.5,'/content/sample_data/aksharantar_sampled.zip','guj')

In [10]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:

# Define hyperparameters to sweep
sweep_config = {
    "method": "bayes",
    'name'  : 'Train Dataset Run',
    'metric': {'goal': 'maximize', 'name': 'Val_Accuracy'},
    "parameters": {
        "input_embedding_size": {"values": [16, 32, 64, 256]},
        "encoder_num_layers": {"values": [1, 2, 3]},
        "decoder_num_layers": {"values": [1, 2, 3]},
        "hidden_size": {"values": [128,256,512,1024]},
        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},
        "bidirectional": {"values": [True, False]},
        "batch_size": {"values": [32, 64]},
        "learning_rate": {"values": [0.001, 0.0001]},
        "num_epochs": {"values": [5,10]},
        "dropout": {"values": [0.2, 0.3]},
        "teacher_forcing_ratio" : {"values":[0.5]},
        "attention": {"values": [False]},
        "beam_width": {"values": [1,2,5,10,20]}
    }
}

# Initialize wandb sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="DL_Assignment_3_CS23M046")






Create sweep with ID: 6fu3q7la
Sweep URL: https://wandb.ai/cs23m046/DL_Assignment_3_CS23M046/sweeps/6fu3q7la


In [12]:
def main_1():

    # Initialize wandb
    with wandb.init() as run:

        config = wandb.config

        run_name=str(config.cell_type)+"_embedding_"+str(config.input_embedding_size)+"_hidden_size_"+str(config.hidden_size)+"_bidirectional_"+str(config.bidirectional)+"_Encoder_layers_"+str(config.encoder_num_layers)+"_Decoder_layers_"+str(config.decoder_num_layers)
        wandb.run.name=run_name

        pathofzip='/content/sample_data/aksharantar_sampled.zip'
        Folder_name='guj'

        arguments(config.input_embedding_size,config.encoder_num_layers,config.decoder_num_layers,config.hidden_size,config.cell_type,config.bidirectional,config.batch_size,config.learning_rate,config.num_epochs,config.dropout,config.teacher_forcing_ratio,config.attention,config.beam_width,pathofzip,Folder_name)



# Run sweep
wandb.agent(sweep_id, function=main_1, count=5)

wandb.finish()

wandb: Agent Starting Run: 044wig7y with config:
wandb: 	attention: False
wandb: 	batch_size: 32
wandb: 	beam_width: 5
wandb: 	bidirectional: False
wandb: 	cell_type: RNN
wandb: 	decoder_num_layers: 3
wandb: 	dropout: 0.2
wandb: 	encoder_num_layers: 2
wandb: 	hidden_size: 128
wandb: 	input_embedding_size: 256
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 5
wandb: 	teacher_forcing_ratio: 0.5
wandb: Currently logged in as: cs23m046. Use `wandb login --relogin` to force relogin


100%|██████████| 128/128 [00:13<00:00,  9.79it/s]



Epoch 1/5,
 Train Loss: 1.1163, Train Accuracy: 0.0000,
 Val Loss: 0.0069, Val Accuracy: 0.0000



100%|██████████| 128/128 [00:12<00:00,  9.88it/s]



Epoch 2/5,
 Train Loss: 0.9090, Train Accuracy: 0.0000,
 Val Loss: 0.0063, Val Accuracy: 0.0000



...


Epoch 9/10,
 Train Loss: 0.3695, Train Accuracy: 0.1045,
 Val Loss: 0.0042, Val Accuracy: 0.0356



100%|██████████| 128/128 [00:13<00:00,  9.73it/s]



Epoch 10/10,
 Train Loss: 0.3445, Train Accuracy: 0.1296,
 Val Loss: 0.0029, Val Accuracy: 0.1265


 Predictions Generated by current sweep and actual output:



,0,1,2
0,INPUT,PREDICTED,ACTUAL
1,vasteena,વસ્તિનના,વસતીના
2,gardanana,ગર્દાનનાા,ગરદનના
3,roqsa,રોપ્સસા,રોક્સ
4,suparastarsa,સુપરસાસ્્્રસા,સુપરસ્ટાર્સ
...,...,...,...
4092,humirsir,હમમિરિસર,હમીરસર
4093,batata,બટટા,બટાટા
4094,dhovaai,ધોવાઈ,ધોવાઇ
4095,daridrataa,દરિદ્રતાા,દરિદ્રતા


Epoch,▁▂▃▃▄▅▆▆▇█
Train_Accuracy,▁▁▁▁▂▃▄▅▇█
Train_Loss,█▆▅▄▃▂▂▂▁▁
Val_Accuracy,▁▁▁▁▂▁▄▅▃█
Val_Loss,█▆▅▅▃▃▂▁▃▁
Epoch,10
Train_Accuracy,0.12957
Train_Loss,0.34454
Val_Accuracy,0.12646
Val_Loss,0.00291
